In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
df.shape

(768, 9)

In [8]:
x=df.drop('Outcome',axis=1)
y=df['Outcome']

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [11]:
y_train.value_counts()

0    395
1    219
Name: Outcome, dtype: int64

In [12]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
x_train,y_train=smote.fit_resample(x_train,y_train)
y_train.value_counts()

1    395
0    395
Name: Outcome, dtype: int64

In [13]:
from sklearn.model_selection import ShuffleSplit
cv=ShuffleSplit(n_splits=10,test_size=.2,random_state=0)


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier


In [15]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    #'naive_bayes_multinomial': {
     #   'model': MultinomialNB(),
      #  'params': {}
    #},
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
            
        }
    }     
}


In [16]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.807595,"{'C': 20, 'kernel': 'rbf'}"
1,random_forest,0.794304,{'n_estimators': 10}
2,logistic_regression,0.738608,{'C': 5}
3,naive_bayes_gaussian,0.720253,{}
4,decision_tree,0.753165,{'criterion': 'gini'}


In [17]:
model_params = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params' : {'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
    },
        'XGBClassifier': {
            'model':XGBClassifier(),
            'params' : {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
            }
        }
}

In [18]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,KNN,0.808228,"{'metric': 'minkowski', 'n_neighbors': 5, 'wei..."
1,XGBClassifier,0.813291,"{'colsample_bytree': 1.0, 'gamma': 1, 'max_dep..."


In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
model_params ={
    'BaggingClassifier':{
        'model' :BaggingClassifier(),
        'params':{
        'base_estimator' : [DecisionTreeClassifier(random_state=0)],
        'n_estimators' : [50,100,200],
        'max_samples': [0.5,0.9,1]}
        
        
    }
    
}

In [20]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,BaggingClassifier,0.83038,{'base_estimator': DecisionTreeClassifier(rand...


In [23]:
model_params ={
    'BaggingClassifier':{
        'model' :BaggingClassifier(),
        'params':{
        'base_estimator' : [XGBClassifier()],
        'n_estimators' : [50,100,200],
        'max_samples': [0.5,0.9,1]}
        
        
    }
    
}

In [24]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,BaggingClassifier,0.801899,"{'base_estimator': XGBClassifier(), 'max_sampl..."


In [26]:
model=BaggingClassifier(base_estimator= DecisionTreeClassifier(random_state=0), max_samples= 0.9, n_estimators= 50)


In [27]:
model.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=0),
                  max_samples=0.9, n_estimators=50)